In [ ]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from rlx.ml import Batches, show_image_mosaic
from rlx.utils import humanbytes
from time import time
import tflearn, psutil, gc
import h5py
from datetime import datetime
%matplotlib inline

In [ ]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import tflearn
import tensorflow as tf
print "free mem", humanbytes(psutil.virtual_memory().free)

## Exercise 1: Load CIFAR-10 and create `info` data

### Part A: Load CIFATR10
download the dataset from https://www.cs.toronto.edu/~kriz/cifar.html and create a function to return selected batches 


In [ ]:
def load_cifar(batches = [1,2,3,4,5]):

    # --------------------------
    # YOUR CODE HERE
    # --------------------------
    ...
    
    return imgs, labels, ohlabs

def train_test_split(imgs, labels, ohlabs, train_pct=.8, shuffle=True):

    # --------------------------
    # YOUR CODE HERE
    # --------------------------
    ...

    return train_imgs, train_labels, train_ohlabs, test_imgs, test_labels, test_ohlabs

** we will use only batch 1**. create variables as in previous problemsets and create additional **INFO** variables with the new information.

data shapes must be as follows:

        imgs         (10000, 32, 32, 3)
        labels       (10000,)
        onehot       (10000, 10)
        train_imgs   (8000, 32, 32, 3)
        train_labels (8000,)
        train_ohlabs (8000, 10)
        train_info   (8000,)
        test_imgs    (2000, 32, 32, 3)
        test_labels  (2000,)
        test_ohlabs  (2000, 10)
        test_info    (2000,)

In [ ]:
imgs, labels, ohlabs = load_cifar(batches = [1])
d = train_test_split(imgs, labels, ohlabs,)
train_imgs, train_labels, train_ohlabs, test_imgs, test_labels, test_ohlabs = d

print "imgs  ", imgs.shape
print "labels", labels.shape
print "onehot", ohlabs.shape
print "train_imgs  ", train_imgs.shape
print "train_labels", train_labels.shape
print "train_ohlabs", train_ohlabs.shape
print "test_imgs   ", test_imgs.shape
print "test_labels ", test_labels.shape
print "test_ohlabs ", test_ohlabs.shape
gc.collect()
print "free mem", humanbytes(psutil.virtual_memory().free)

### Part B: Generate info

we will add **ONE NEURON** at the last **fully connected layer** with the following values:

- **+1** if the image corresponds to a **LIVING THING**
- **-1** otherwise

complete the following function to transform the `labels` numpy array passed as argument using this convention, such as

    [3, 1, 7, 9] --> [1, -1, 1, -1]

In [ ]:
labcode= np.r_[[  0    ,   1,     2,      3,     4,      5,      6,      7,       8,      9   ]]
cnames = np.r_[["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "boat", "truck"]]
info   = np.r_[[ -1,      -1,     1,      1,     1,      1,      1,      1,       -1,     -1  ]]

def get_info(labels):
    
    # --------------------------
    # YOUR CODE HERE
    # --------------------------
    labels_info = ...
    
    return labels_info

so now we create the info variables for train and test

In [ ]:
train_info = get_info(train_labels)
test_info  = get_info(test_labels)

## Exercise 2: Build `tflearn` model

use the same network as in the previous problemset **BUT** add an additional input neuron at OUTPUT layer:

| layer   | input_size  | output_size | filter_size  | stride | n_filters |activation| var sizes  | params |
| ------- |:-----------:|:-----------:|:------------:|:------:|:---------:|:--------:|:--------------:| |
| conv1   | 32x32x3     | 32x32x9     | 5x5          |1       | 15        | relu     | W1 = [5,5,3,15]<br/> b = [15]|-|
| conv2   | 32x32x15    | 16x16x18    | 5x5          |2       | 18        | relu     | W2 = [5,5,15,18]<br/> b = [18]|-|
| conv3   | 16x16x18    | 8x8x20      | 3x3          |2       | 20        | relu     | W2 = [3,3,18,20]<br/> b =[20]|-|
| maxpool | 8x8x20      | 4x4x20      |              |        |           |          | | k = 2 |
| fc      | 4x4x20      |    100      |              |        |           | relu     | W3 = [320,100]<br/>b=[100]|-|
| dropout | 100         |   100       |              |        |           |          | | pkeep = .75 |
| output **+ INFO**  | **101**         |   10        |              |        |           | softmax  | W4 = [**101**,10]<br/>b=[10]|-|


In [ ]:
def get_TF_graph():
    tf.reset_default_graph()
    
    with tf.name_scope("data"):
        # --------------------------
        # YOUR CODE HERE
        # --------------------------
        X      = ... 
        X_info = ...
        
        Y      = ...
        network = tf.reshape(X, [-1, 32, 32, 3])
    
    with tf.name_scope("layers"):
        # --------------------------
        # YOUR CODE HERE
        # --------------------------

        num_classes=10
        network = ...
        network = ...
        network = ...
        y_hat   = ...

    with tf.name_scope("loss"):
        # --------------------------
        # YOUR CODE HERE
        # --------------------------
        cross_entropy = ...
        loss          = tf.reduce_mean(cross_entropy)*100

    with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.argmax(y_hat, 1), tf.argmax(Y, 1))
        accuracy     = tf.reduce_mean(tf.cast(correct_pred, tf.float32))    
    
    with tf.name_scope("optimizer"):
        # --------------------------
        # YOUR CODE HERE
        # --------------------------
        train_step = ...
        
    return X, Y, X_info, y_hat, loss, train_step, accuracy

print "free mem", humanbytes(psutil.virtual_memory().free)

In [ ]:
X, Y, X_info, y_hat, loss, train_step, accuracy = get_TF_graph()
vars = {i.name:i for i in tflearn.variables.get_all_trainable_variable()}
vars

## Exercise 3: implement optimization loop

In [ ]:
model_name = "cnn_cifar10_info_" + datetime.now().strftime("%m-%d_%H:%M")
print model_name

In [ ]:
def fit (X, Y, X_info,
         X_train, y_train, X_train_info, X_test, y_test, X_test_info,
         model_name, loss, train_step, accuracy, 
         batch_size, n_epochs, log_freq):
    
    # --------------------------
    # YOUR CODE HERE
    # --------------------------
    ...
    
    return log_train, log_test, model_name

def plot_results(log_train, log_test):
    k = log_train.rolling(window=10).mean().dropna()
    plt.plot(k.time, k.accuracy, color="blue", lw=2, label="train")
    plt.plot(log_test.time, log_test.accuracy, color="red",lw=2, label="test")
    plt.legend(loc="center left", bbox_to_anchor=(1,.5))
    plt.plot(log_train.time, log_train.accuracy, alpha=.3, color="blue")
    plt.grid()
    plt.xlabel("elapsed time (secs)")
    plt.ylabel("accuracy")
    plt.axhline(0.5, color="black")
    plt.xlim(0,log_train.time.max()+1)
    plt.title("final train_acc=%.4f, test_acc=%.4f"%(log_train.accuracy.values[-1], log_test.accuracy.values[-1]))

### train your model !!!

In [ ]:
X, Y, X_info, y_hat, loss, train_step, accuracy = get_TF_graph()
print "free mem", humanbytes(psutil.virtual_memory().free)

In [ ]:
log_train, log_test, model_name = fit(X, Y, X_info,
                                      train_imgs, train_ohlabs, train_info,
                                      test_imgs, test_ohlabs, test_info,
                                      "extended_cnn_mnist", loss, train_step, accuracy,
                                      batch_size=100, n_epochs=15, log_freq=80)

plot_results(log_train, log_test)
print "free mem", humanbytes(psutil.virtual_memory().free)

## Exercise 4: visualize filters, sample misses and activations

In [ ]:
# --------------------------
# YOUR CODE HERE
# --------------------------
